In [1]:
import pandas as pd

In [3]:
#读取附件1 企业信息
data1=pd.read_excel('附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name='企业信息')
data1.head()

Wall time: 3.06 s


,企业代号,企业名称,信誉评级,是否违约
0,E1,***电器销售有限公司,A,否
1,E2,***技术有限责任公司,A,否
2,E3,***电子(中国)有限公司***分公司,C,否
3,E4,***发展有限责任公司,C,否
4,E5,***供应链管理有限公司,B,否


In [4]:
#读取附件1 进项发票信息
data2=pd.read_excel('附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name='进项发票信息')
data2.head()

,企业代号,发票号码,开票日期,销方单位代号,金额,税额,价税合计,发票状态
0,E1,3390939,2017-07-18,A00297,-943.40,-56.60,-1000.00,有效发票
1,E1,3390940,2017-07-18,A00297,-4780.24,-286.81,-5067.05,有效发票
2,E1,3390941,2017-07-18,A00297,943.40,56.60,1000.00,有效发票
3,E1,3390942,2017-07-18,A00297,4780.24,286.81,5067.05,有效发票
4,E1,9902669,2017-08-07,A05061,326.21,9.79,336.00,有效发票


In [5]:
#读取附件1 销项发票信息
data3=pd.read_excel('附件1：123家有信贷记录企业的相关数据.xlsx',sheet_name='销项发票信息')
data3.head()

,企业代号,发票号码,开票日期,购方单位代号,金额,税额,价税合计,发票状态
0,E1,11459356,2017-08-04,B03711,9401.71,1598.29,11000.0,有效发票
1,E1,5076239,2017-08-09,B00844,8170.94,1389.06,9560.0,有效发票
2,E1,5076240,2017-08-09,B00844,8170.94,1389.06,9560.0,有效发票
3,E1,5076241,2017-08-09,B00844,4085.47,694.53,4780.0,有效发票
4,E1,5076242,2017-08-09,B00844,4085.47,694.53,4780.0,有效发票


In [6]:
def filter_extreme_MAD(dataframe,n=3):
    # MAD:绝对中位差去极值
    for i in dataframe.columns:
        median = dataframe[i].quantile(0.5)
        new_median = ((dataframe[i] - median).abs()).quantile(0.50)
        max_range = median + n * new_median
        min_range = median - n * new_median
        dataframe[i]=pd.DataFrame(np.clip(dataframe[i].values, min_range, max_range),columns=None)

    return dataframe
def filter_extreme_3sigma(dataframe,n=3):
    # 3sigma 去极值
    for i in dataframe.columns:
        mean=dataframe[i].mean()
        std=dataframe[i].std()
        max_range=mean+n*std
        min_range=mean-n*std
        dataframe[i] = pd.DataFrame(np.clip(dataframe[i].values, min_range, max_range), columns=None)
    return dataframe

def filter_extreme_percentile(dataframe,min=0.025,max=0.975):
    # 百分位法 去极值
    for i in dataframe.columns:
        Temp=dataframe[i].sort_values()
        q=Temp.quantile([min,max])
        dataframe[i] = pd.DataFrame(np.clip(dataframe[i].values, q.iloc[0], q.iloc[1]), columns=None)
    return dataframe

In [8]:
#对data2进行异常值与缺失值处理
for i in data2.columns:
    #对数据进行异常值处理，若报错（即数据为文本或者非定量变量），则自动跳过
    try:
        data2[i]=filter_extreme_3sigma(data2[i],n=3)
    except:
        pass
    
#对data3进行异常值与缺失值处理
for i in data3.columns:
    #对数据进行异常值处理，若报错（即数据为文本或者非定量变量），则自动跳过
    try:
        data3[i]=filter_extreme_3sigma(data3[i],n=3)
    except:
        pass        

In [17]:
#以企业代号为分组项，对'金额', '税额', '价税合计'这三个变量提取计数、均值、标准差、最小值、1/4位数、1/2位数、3/4位数，最大值构造特征
#可以看到，从3个变量衍生成了24个变量
data2_huizng_0=data2.groupby('企业代号')[['金额', '税额', '价税合计']].describe()
data2_huizng_0.head()

金额                                                             \
       count          mean           std         min       25%      50%   
企业代号                                                                      
E1    3441.0  1.733225e+06  3.917948e+06 -9823008.85  265.4900  970.870   
E10   4097.0  1.502911e+03  4.137848e+03   -47209.43  190.2900  355.240   
E100    58.0  3.632552e+02  5.203146e+02        2.91  172.4100  197.745   
E101    16.0  2.868861e+03  8.745736e+03      177.36  205.8225  212.650   
E102   730.0  4.413846e+03  7.621901e+03   -14879.93  247.9650  893.810   

                                  税额                 ...              \
             75%         max   count           mean  ...         75%   
企业代号                                                 ...               
E1    98201.7700  9999984.51  3441.0  269882.725539  ...  12766.2300   
E10     969.9000    58685.27  4097.0      78.394757  ...     55.1700   
E100    254.1475     2465.49    58.0      38.865862  ...     29.1125   
E101    264.1500    34883.62    16.0     453.514375  ...     35.8750   
E102   5479.1600    56026.13   730.0     666.817082  ...    746.9325   

                    价税合计                                                  \
             max   count          mean           std          min    25%   
企业代号                                                                       
E1    1699990.41  3441.0  2.003108e+06  4.538253e+06 -11100000.00  300.0   
E10      8275.86  4097.0  1.581306e+03  4.353305e+03    -50042.00  203.0   
E100      320.51    58.0  4.021210e+02  5.803828e+02         3.00  200.0   
E101     5581.38    16.0  3.322375e+03  1.014682e+04       188.00  235.0   
E102     8964.17   730.0  5.080663e+03  8.828317e+03    -17409.44  280.0   

                                        
         50%          75%          max  
企业代号                                    
E1    1000.0  110968.0000  11699934.00  
E10    388.0     999.0000     60445.85  
E100   217.5     275.0000      2786.00  
E101   244.5     280.0000     40465.00  
E102  1012.0    6375.8325     64990.30  

[5 rows x 24 columns]

In [18]:
#根据发票状态变量，可以衍生出特征，发票状态个数，去重值，频数最高的发票状态类别，频数最高的发票状态类别对应频数
#可以看到，又衍生成了4个变量
data2_huizng_1=data2.groupby('企业代号')[['发票状态']].describe()
data2_huizng_1

发票状态                   
     count unique   top  freq
企业代号                         
E1    3441      2  有效发票  3249
E10   4097      2  有效发票  3860
E100    58      1  有效发票    58
E101    16      1  有效发票    16
E102   730      2  有效发票   710
...    ...    ...   ...   ...
E95      4      1  有效发票     4
E96     11      1  有效发票    11
E97      9      1  有效发票     9
E98     51      2  有效发票    50
E99    880      2  有效发票   871

[123 rows x 4 columns]

In [19]:
#将构造好的两个特征列表合并为data2_huizng
data2_huizng=pd.concat([data2_huizng_0,data2_huizng_1] ,axis=1)
data2_huizng

金额                                                              \
       count          mean           std         min       25%       50%   
企业代号                                                                       
E1    3441.0  1.733225e+06  3.917948e+06 -9823008.85  265.4900   970.870   
E10   4097.0  1.502911e+03  4.137848e+03   -47209.43  190.2900   355.240   
E100    58.0  3.632552e+02  5.203146e+02        2.91  172.4100   197.745   
E101    16.0  2.868861e+03  8.745736e+03      177.36  205.8225   212.650   
E102   730.0  4.413846e+03  7.621901e+03   -14879.93  247.9650   893.810   
...      ...           ...           ...         ...       ...       ...   
E95      4.0  2.683950e+02  8.490000e+00      264.15  264.1500   264.150   
E96     11.0  2.108183e+05  6.965140e+05       50.00  245.2850   330.190   
E97      9.0  2.083856e+03  1.389139e+03      264.15  283.0100  2830.190   
E98     51.0  3.912026e+03  1.362694e+04    -1706.84  264.1500   846.020   
E99    880.0  7.255586e+03  6.311452e+04   -89557.52  176.9900   297.455   

                                  税额                 ...          价税合计  \
             75%         max   count           mean  ...           std   
企业代号                                                 ...                 
E1    98201.7700  9999984.51  3441.0  269882.725539  ...  4.538253e+06   
E10     969.9000    58685.27  4097.0      78.394757  ...  4.353305e+03   
E100    254.1475     2465.49    58.0      38.865862  ...  5.803828e+02   
E101    264.1500    34883.62    16.0     453.514375  ...  1.014682e+04   
E102   5479.1600    56026.13   730.0     666.817082  ...  8.828317e+03   
...          ...         ...     ...            ...  ...           ...   
E95     268.3950      281.13     4.0      16.105000  ...  9.000000e+00   
E96     526.3000  2310881.90    11.0   10588.053636  ...  7.313244e+05   
E97    2830.1900     3773.58     9.0     125.033333  ...  1.472485e+03   
E98    2347.2950    97087.38    51.0     195.600588  ...  1.404620e+04   
E99     464.8275   909090.91   880.0     701.254659  ...  6.939586e+04   

                                                            发票状态               \
              min    25%     50%          75%          max count unique   top   
企业代号                                                                            
E1   -11100000.00  300.0  1000.0  110968.0000  11699934.00  3441      2  有效发票   
E10     -50042.00  203.0   388.0     999.0000     60445.85  4097      2  有效发票   
E100         3.00  200.0   217.5     275.0000      2786.00    58      1  有效发票   
E101       188.00  235.0   244.5     280.0000     40465.00    16      1  有效发票   
E102    -17409.44  280.0  1012.0    6375.8325     64990.30   730      2  有效发票   
...           ...    ...     ...          ...          ...   ...    ...   ...   
E95        280.00  280.0   280.0     284.5000       298.00     4      1  有效发票   
E96         50.00  260.0   350.0     566.5000   2426426.00    11      1  有效发票   
E97        280.00  300.0  3000.0    3000.0000      4000.00     9      1  有效发票   
E98      -1997.00  280.0   911.2    2577.4800    100000.00    51      2  有效发票   
E99    -101200.00  200.0   340.0     520.0000   1000000.00   880      2  有效发票   

            
      freq  
企业代号        
E1    3249  
E10   3860  
E100    58  
E101    16  
E102   710  
...    ...  
E95      4  
E96     11  
E97      9  
E98     50  
E99    871  

[123 rows x 28 columns]

In [21]:
#将data2_huizng与data1进行合并，以企业代号为连接键
union_data=pd.merge(data1,data2_huizng,left_on='企业代号',right_on='企业代号')
union_data

,企业代号,企业名称,信誉评级,是否违约,"(金额, count)","(金额, mean)","(金额, std)","(金额, min)","(金额, 25%)","(金额, 50%)",...,"(价税合计, std)","(价税合计, min)","(价税合计, 25%)","(价税合计, 50%)","(价税合计, 75%)","(价税合计, max)","(发票状态, count)","(发票状态, unique)","(发票状态, top)","(发票状态, freq)"
0,E1,***电器销售有限公司,A,否,3441.0,1.733225e+06,3.917948e+06,-9823008.85,265.4900,970.870,...,4.538253e+06,-11100000.00,300.0000,1000.00,110968.000,11699934.00,3441,2,有效发票,3249
1,E2,***技术有限责任公司,A,否,32156.0,5.117059e+03,2.511614e+04,-99857.14,172.4200,297.345,...,2.593555e+04,-100000.00,200.0000,330.00,810.000,1000000.00,32156,2,有效发票,31435
2,E3,***电子(中国)有限公司***分公司,C,否,4561.0,1.210060e+04,5.457243e+04,-321045.22,211.6500,367.930,...,5.697186e+04,-321045.22,227.0000,392.00,1756.000,802807.74,4561,2,有效发票,4367
3,E4,***发展有限责任公司,C,否,558.0,5.881122e+05,2.001187e+06,-3798756.81,319.1175,938.690,...,2.328125e+06,-4406557.90,360.2500,982.85,35000.000,10980265.78,558,2,有效发票,521
4,E5,***供应链管理有限公司,B,否,2169.0,9.310316e+04,3.316620e+05,-98909.09,389.3800,80653.210,...,3.840980e+05,-108800.00,414.0000,89244.00,113751.340,7536626.40,2169,2,有效发票,2084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,E119,***药房,D,是,315.0,6.977528e+02,1.228014e+03,-34.51,177.0800,443.770,...,1.383500e+03,-39.00,200.1000,504.44,858.625,14285.80,315,2,有效发票,314
119,E120,***陈列广告有限公司,D,是,36.0,7.849186e+02,2.312309e+03,26.54,129.3100,172.410,...,2.448252e+03,27.34,150.0000,200.00,258.250,10750.00,36,2,有效发票,35
120,E121,***药业连锁有限公司***药店,D,是,50.0,2.146972e+04,2.587003e+04,-2264.15,317.3925,2264.150,...,3.027725e+04,-2400.00,331.4025,2400.00,58857.530,75608.91,50,1,有效发票,50
121,E122,***商贸有限责任公司,D,是,48.0,1.074576e+03,2.850091e+03,-4817.24,168.0750,264.150,...,3.056484e+03,-5588.00,194.0000,280.00,962.000,18694.26,48,2,有效发票,47


In [22]:
#同理处理data3

data3_huizng=data3.groupby('企业代号')[['金额', '税额', '价税合计']].describe()

data3_huizng=pd.concat([data3_huizng,data3.groupby('企业代号')[['发票状态']].describe()] ,axis=1)

#将union_data与data3_huizng合并
union_data=pd.merge(union_data,data3_huizng,left_on='企业代号',right_on='企业代号')
union_data.shape

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


(123, 60)

In [24]:
#因为构造好的特征需要与附件2保持一致，因此剔除变量信誉评级
del union_data['信誉评级']

In [25]:
#可以看到，一共有123家企业数据，59-3项特征（X）,Y是【是否违约】
union_data.head()

,企业代号,企业名称,是否违约,"('金额', 'count')_x","('金额', 'mean')_x","('金额', 'std')_x","('金额', 'min')_x","('金额', '25%')_x","('金额', '50%')_x","('金额', '75%')_x",...,"(价税合计, std)","(价税合计, min)","(价税合计, 25%)","(价税合计, 50%)","(价税合计, 75%)","(价税合计, max)","(发票状态, count)","(发票状态, unique)","(发票状态, top)","(发票状态, freq)"
0,E1,***电器销售有限公司,否,3441.0,1.733225e+06,3.917948e+06,-9823008.85,265.4900,970.870,98201.77,...,493962.263666,-1145690.00,122176.500,600256.75,1096494.00,1169977.00,8110,2,有效发票,7886
1,E2,***技术有限责任公司,否,32156.0,5.117059e+03,2.511614e+04,-99857.14,172.4200,297.345,766.99,...,46052.957199,-100000.00,9012.000,60000.00,100000.00,100000.00,12707,2,有效发票,11665
2,E3,***电子(中国)有限公司***分公司,否,4561.0,1.210060e+04,5.457243e+04,-321045.22,211.6500,367.930,1655.66,...,120731.750911,-1127351.73,1680.000,3907.00,22569.62,1160953.50,24073,2,有效发票,23688
3,E4,***发展有限责任公司,否,558.0,5.881122e+05,2.001187e+06,-3798756.81,319.1175,938.690,33980.58,...,271272.684844,-1169732.68,1125048.355,1151000.00,1166400.00,1169980.44,2231,2,有效发票,2041
4,E5,***供应链管理有限公司,否,2169.0,9.310316e+04,3.316620e+05,-98909.09,389.3800,80653.210,99115.04,...,314360.510764,-115000.00,112100.000,115430.00,115913.00,1159532.60,1060,2,有效发票,1005


In [26]:
#将附件1数据保存下来
union_data.to_csv('cleandata附件1.csv',index=None,encoding='gbk')